In [1]:
import yaml
def load_config(yaml_file):
    with open(yaml_file, "r") as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")

cuda_devices = config["cuda"]["devices"]
# data
data_dir = config["data"]
# model
model_name = config["model"]["name"]
#lora
rank = int(config["lora"]["rank"])
alpha = int(config["lora"]["alpha"])

# training
max_seq_length = config["training"]["max_seq_length"]
learning_rate = float(config["training"]["learning_rate"])
warmup_steps = int(config["training"]["warmup_steps"])
lr_scheduler_type = str(config["training"]["lr_scheduler_type"])
train_epochs = config["training"]["train_epochs"]
per_device_batch_size = config["training"]["per_device_batch_size"]
gradient_accumulation_steps = config["training"]["gradient_accumulation_steps"]
save_steps = config["training"]["save_steps"]
eval_steps = config["training"]["eval_steps"]
logging_steps = config["training"]["logging_steps"]
random_seed = config["training"]["random_seed"]

In [2]:
### set the cuda device(s)
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = cuda_devices
print("cuda devices:", cuda_devices)

cuda devices: 0


In [3]:
### load libraries and model from HF 
import torch
import pandas as pd
from functools import partial
import wandb
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM # to train model only on the generated prompts
#from trl.trainer import ConstantLengthDataset
from datasets import load_dataset
from _1_prompt_temp_v1 import instruction_format, conversational_format, sys_prompt
from unsloth import FastLanguageModel, is_bfloat16_supported, apply_chat_template

timestamp = pd.Timestamp.now().strftime("%Y%m%d%H%M")
# Initialize WandB (ensure you've logged in using `wandb login`)
wandb.init(project="code-llama-finetuning", 
           name=f"fine-tune-{model_name.split('/')[-1]}-{data_dir.split('/')[-1]}_{timestamp}",
           config={"learning_rate": learning_rate, "num_train_epochs": train_epochs, "max_seq_length": max_seq_length, "learning_rate": learning_rate})

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: Currently logged in as: priscillachyrva (priscillachyrva-university-mannheim). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# Model configuration

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=rank,
    lora_alpha=alpha,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state= random_seed
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

codellama/CodeLlama-7b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data

In [8]:
dataset = load_dataset(data_dir)

for split in dataset:
    dataset[split] = dataset[split].map(lambda x: conversational_format(x, include_description=True, include_ascii=True))

#train_dataset, val_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]

# Print out a sample to confirm the changes
#print(train_dataset["conversations"][0])

In [12]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "alpaca", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    #mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [15]:
print(train_dataset["conversations"][0])

[{'content': 'Generate a python program producing the graphic, which is described and depicted as follows:\n The Program draws a 5 pointed star\n Graphic:\n00000000000000000000000000000000000\n00000000000000000000000120000000000\n00000000000000000000002220000000000\n00000000000000000000031020000000000\n00000000000000000001300002000000000\n00000000000000000021000002000000000\n00000000000000000300000002000000000\n00000000000000012000000000200000100\n00000000000000310000000000300000200\n00000000000013000000000000200001100\n00000000000220000000000000020002000\n00000000003100000000000000020002000\n00000000130000000000000000020001000\n00000002100000000000000000002020000\n00000030000000000000000000002020000\n00001200000000000000000000002020000\n00031000000000000000000000000500000\n00400000000000000000000000000400000\n00031000000000000000000000000500000\n00001200000000000000000000002020000\n00000030000000000000000000002020000\n00000002100000000000000000002020000\n000000001300000000000000000200

In [ ]:
print(train_dataset["text"][0])

In [13]:
dataset = dataset.map(formatting_prompts_func, batched = True,)
train_dataset, val_dataset, test_dataset = dataset["train"], dataset["validation"], dataset["test"]

# Print out a sample to confirm the changes
print(train_dataset["text"][0])

Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

<s>Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
Generate a python program producing the graphic, which is described and depicted as follows:
 The Program draws a 5 pointed star
 Graphic:
00000000000000000000000000000000000
00000000000000000000000120000000000
00000000000000000000002220000000000
00000000000000000000031020000000000
00000000000000000001300002000000000
00000000000000000021000002000000000
00000000000000000300000002000000000
00000000000000012000000000200000100
00000000000000310000000000300000200
00000000000013000000000000200001100
00000000000220000000000000020002000
00000000003100000000000000020002000
00000000130000000000000000020001000
00000002100000000000000000002020000
00000030000000000000000000002020000
00001200000000000000000000002020000
00031000000000000000000000000500000
00400000000000000000000000000400000
00031000000000000000000000000500000
00001200000000000000000000002020000
00000030

In [12]:
### define and apply chat template
chat_template = """{SYSTEM}
### Instruction:
{INPUT}
### Python Program:
{OUTPUT}"""


dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    #default_system_message = sys_prompt, # optional
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/8006 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

Map:   0%|          | 0/997 [00:00<?, ? examples/s]

In [14]:
#print(dataset["train"]["conversations"][0])
print(dataset["train"]["text"][0])

<s>Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
Generate a python program producing the graphic, which is described and depicted as follows:
 The Program draws a 5 pointed star
 Graphic:
00000000000000000000000000000000000
00000000000000000000000120000000000
00000000000000000000002220000000000
00000000000000000000031020000000000
00000000000000000001300002000000000
00000000000000000021000002000000000
00000000000000000300000002000000000
00000000000000012000000000200000100
00000000000000310000000000300000200
00000000000013000000000000200001100
00000000000220000000000000020002000
00000000003100000000000000020002000
00000000130000000000000000020001000
00000002100000000000000000002020000
00000030000000000000000000002020000
00001200000000000000000000002020000
00031000000000000000000000000500000
00400000000000000000000000000400000
00031000000000000000000000000500000
00001200000000000000000000002020000
00000030

# Training configuration

In [15]:
model_name_short = model_name.split("/")[-1]
data_short = data_dir.split("/")[-1]
trainer_1 = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field = "text", # used as model input
    compute_metrics=None, # (Callable[[transformers.EvalPrediction], dict], optional defaults to None) — The function used to compute metrics during evaluation. It should return a dictionary mapping metric names to metric values. If not specified, only the loss will be computed during evaluation.
    args = SFTConfig(
        output_dir = f"./results/fine-tune-{model_name.split('/')[-1]}-{data_dir.split('/')[-1]}",
        # pre-processing
        max_seq_length = max_seq_length,
        packing = False,
        dataset_num_proc =4,
        # training parameters
        learning_rate = learning_rate,
        warmup_steps = warmup_steps,
        lr_scheduler_type = lr_scheduler_type,
        num_train_epochs = train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        per_device_train_batch_size = per_device_batch_size,
        per_device_eval_batch_size = per_device_batch_size,
        # reporting and logging
        report_to = ["wandb"],
        push_to_hub = True,
        hub_model_id = f"fine-tune-{model_name.split('/')[-1]}-{data_dir.split('/')[-1]}",
        logging_strategy = "steps",
        logging_steps = logging_steps,
        # checkpointing
        save_strategy = "steps",
        save_steps = save_steps,
        # evaluation
        eval_strategy = "steps",
        eval_steps = eval_steps,
        # optimization
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),    
        # ensure reproducibility
        seed = random_seed,
        data_seed = random_seed,
        full_determinism=True
        )
    )

## Training on completions only ###
https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only

orients on the instruction_format_v1 function from _1_prompt_temp_v2.py but excludes the include_description and include_ascii arguments

In [ ]:
#def formatting_prompts_func(example):
#    output_texts = []
#    for i in range(len(example["Description"])):
#        formated_input = f"Generate a python program producing the graphic, which is described and depicted as follows:\n    The Program draws {example['Description'][i]}\n    Graphic:\n{example['ASCII-Art'][i]}\n"
#        text = f"### Instruction: {formated_input}\n### Python Program: {example['Program'][i]}"
#        output_texts.append(text)
#    return output_texts


response_template = "### Response::" 
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer_2 = SFTTrainer(
    model,
    data_collator=collator,          
    formatting_func=formatting_prompts_func, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=None, # (Callable[[transformers.EvalPrediction], dict], optional defaults to None) — The function used to compute metrics during evaluation. It should return a dictionary mapping metric names to metric values. If not specified, only the loss will be computed during evaluation.
    args = SFTConfig(
        output_dir = f"./results/fine-tune-{model_name.split('/')[-1]}-{data_dir.split('/')[-1]}",
        # pre-processing
        max_seq_length = max_seq_length,
        packing = False,
        dataset_num_proc =4,
        # training parameters
        learning_rate = learning_rate,
        warmup_steps = warmup_steps,
        lr_scheduler_type = lr_scheduler_type,
        num_train_epochs = train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        per_device_train_batch_size = per_device_batch_size,
        per_device_eval_batch_size = per_device_batch_size,
        # reporting and logging
        report_to = ["wandb"],
        push_to_hub = True,
        hub_model_id = f"fine-tune-{model_name.split('/')[-1]}-{data_dir.split('/')[-1]}",
        logging_strategy = "steps",
        logging_steps = logging_steps,
        # checkpointing
        save_strategy = "steps",
        save_steps = save_steps,
        # evaluation
        eval_strategy = "steps",
        eval_steps = eval_steps,
        # optimization
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),    
        # ensure reproducibility
        seed = random_seed,
        data_seed = random_seed,
        full_determinism=True
        )
    )

In [16]:
trainer_1.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,006 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 4
\        /    Total batch size = 256 | Total steps = 31
 "-____-"     Number of trainable parameters = 19,988,480
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 752.00 MiB. GPU 0 has a total capacity of 47.53 GiB of which 174.81 MiB is free. Process 3306257 has 7.07 GiB memory in use. Including non-PyTorch memory, this process has 40.21 GiB memory in use. Of the allocated memory 39.72 GiB is allocated by PyTorch, and 189.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
if config["training"]["trainer_output_only"]:
    trainer_2.train()
else:
    trainer_1.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,006 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 4
\        /    Total batch size = 256 | Total steps = 31
 "-____-"     Number of trainable parameters = 19,988,480
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


KeyboardInterrupt: 